In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Download and Explore the Dataset

# 1-1.Seoul

- To get longitude and latitude information for each area, get data from a Korean govenment site(https://www.data.go.kr/dataset/3045281/fileData.do). There are 24 files that include addresses and coordinates of each area of 25 districts in Seoul.For your convenience, I downloaded those 24 files(1 file is erroneous), made them as 1 file, and placed .csv file on the server, so you can simply use it.

- The coordinates information in the datafile has KR2000 system. We need to change it to wgs84 system. Let's import package to covert the coordinate systems. We conterted KR2000 system to wgs84 and many of the converted coordinates are incorrect. It maybe because of either original data error or coordinates' system conversion library error.

In [9]:
df=pd.read_csv('Seoul7.csv')
df.head()
df=df.drop('Unnamed: 2',1)
df.columns=['District','Area']
print('Total record is',df.shape)
df.head()
#df.groupby(['District']).count().reset_index()

Total record is (330, 2)


,District,Area
0,Jungnang-gu,Mangu-dong
1,Jungnang-gu,Myeonmok-dong
2,Jungnang-gu,Muk-dong
3,Jungnang-gu,Sangbong-dong
4,Jungnang-gu,Sinnae-dong


In [10]:
geo_df=pd.read_csv('Seoul_loc.csv')
geo_df=geo_df[['Area', 'col4', 'col5']]
geo_df.head()

,Area,col4,col5
0,Mangu-dong,208891.8729,455286.4398
1,Myeonmok-dong,207637.5865,453866.2859
2,Muk-dong,206895.5941,456880.7068
3,Sangbong-dong,207588.3858,455220.8446
4,Sinnae-dong,208719.1540,456525.1101


In [2]:
!pip install pyshp
!pip install pyproj 

In [3]:
import shapefile  #the pyshp module : Should install pyshp module.
from pyproj import Proj, transform  #Should install pyproj module.

In [42]:
# Change coordinate system
# korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
inProj = Proj(init = 'epsg:5186')
outProj= Proj(init = 'epsg:4326')

x,y =206895.5941, 456880.7068
long_,lat_ = transform(inProj,outProj,x,y)
print(lat_,long_)
# latitude = []
# longitude= []
# for idx,row in df_node.iterrows():
#     x,y  = row.coords[0][0],row.coords[0][1]  # korea 2000 좌표계
#     nx,ny = transform(inProj,outProj,x,y)     # 새로운 좌표계    
#     latitude.append(ny)
#     longitude.append(nx)
# df_node['latitude'] = latitude
# df_node['longitude']= longitude
# del df_node['coords'] # delete coords

36.710429872638706 127.07717674509125


###  Here, I use Nominatim geolocation function to get coordinate instead. Seoul has 24 districts

In [11]:
Seq_dist=df['District'].unique()
Seq_dist

array(['Jungnang-gu', 'Gangnam-gu', 'Gangdong-gu', 'Gangbuk-gu',
       'Gangseo-gu', 'Gwanak-gu', 'Gwangjin-gu', 'Guro-gu',
       'Geumcheon-gu', 'Nowon-gu', 'Dobong-gu', 'Dongdaemun-gu',
       'Mapo-gu', 'Seodaemun-gu', 'Seocho-gu', 'Seongdong-gu',
       'Seongbuk-gu', 'Songpa-gu', 'Yangcheon-gu', 'Yeongdeungpo-gu',
       'Yongsan-gu', 'Eunpyeong-gu', 'Jongno-gu', 'Jung-gu'], dtype=object)

In [12]:
# define the dataframe columns
column_names = ['District', 'Lat', 'Long'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

for item in Seq_dist:
    address = '{}, Seoul'.format(item)

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    neighborhoods = neighborhoods.append({'District': item,
                                          'Lat': latitude,
                                          'Long': longitude}, ignore_index=True)    
neighborhoods

,District,Lat,Long
0,Jungnang-gu,37.597957,127.092666
1,Gangnam-gu,37.500590,127.050775
2,Gangdong-gu,37.547070,127.141903
3,Gangbuk-gu,37.640495,127.009235
4,Gangseo-gu,37.558340,126.832201
5,Gwanak-gu,37.467258,126.948288
6,Gwangjin-gu,37.544748,127.080943
7,Guro-gu,37.498554,126.859183
8,Geumcheon-gu,37.460096,126.901626
9,Nowon-gu,37.651480,127.073182


In [13]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, dist in zip(neighborhoods['Lat'], neighborhoods['Long'], neighborhoods['District']):
    label = '{}, {}'.format('Seoul', dist)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# 1-2.New York City

### New York City has 5 buroughs. And we get coordinates to compare

In [14]:
# define the dataframe columns
column_names = ['Borough', 'Lat', 'Long'] 
# instantiate the dataframe
NY_neighborhoods = pd.DataFrame(columns=column_names)
NY_Seq_dist= ['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island']

for item in NY_Seq_dist:
    address = '{}, NY'.format(item)

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    NY_latitude = location.latitude
    NY_longitude = location.longitude
    
    NY_neighborhoods = NY_neighborhoods.append({'Borough': item,
                                          'Lat': NY_latitude,
                                          'Long': NY_longitude}, ignore_index=True)    
NY_neighborhoods

,Borough,Lat,Long
0,Bronx,40.850485,-73.840404
1,Manhattan,40.790087,-73.959829
2,Brooklyn,40.650104,-73.949582
3,Queens,40.652493,-73.791421
4,Staten Island,40.583456,-74.149605


In [15]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[NY_latitude, NY_longitude], zoom_start=10)

# add markers to map
for lat, lng, dist in zip(NY_neighborhoods['Lat'], NY_neighborhoods['Long'], NY_neighborhoods['Borough']):
    label = '{}, {}'.format('Seoul', dist)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

# 2. Explore Neighborhoods in Seoul

# 2-1.Seoul

#### Define Foursquare Credentials and Version

In [16]:
# CLIENT_ID = 'your-client-ID' # your Foursquare ID
# CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
# VERSION = '20180605' # Foursquare API version
CLIENT_ID = '35QV4AKD11HYDW0QXK1XG2TBIOKNHEO0FB0YPMRG41QXKWMJ' # your Foursquare ID
CLIENT_SECRET = '0VXS0OG3R4HLMXCIEY4UP1UJIFQU40NJPQMWIEKRYKWLIVR1' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 35QV4AKD11HYDW0QXK1XG2TBIOKNHEO0FB0YPMRG41QXKWMJ
CLIENT_SECRET:0VXS0OG3R4HLMXCIEY4UP1UJIFQU40NJPQMWIEKRYKWLIVR1


#### Let's explore the first neighborhood in our dataframe.

In [17]:
neighborhoods.loc[1, 'District']

'Gangnam-gu'

In [18]:
neighborhood_latitude = neighborhoods.loc[1, 'Lat'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[1, 'Long'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[1, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Gangnam-gu are 37.50059, 127.050775.


#### Now, let's get the top 100 venues that are in Gangnam-gu within a radius of 2500 meters.

In [19]:
n_latitude=neighborhood_latitude
n_longitude=neighborhood_longitude
radius =2500
LIMIT=100

In [20]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, n_latitude, n_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=35QV4AKD11HYDW0QXK1XG2TBIOKNHEO0FB0YPMRG41QXKWMJ&client_secret=0VXS0OG3R4HLMXCIEY4UP1UJIFQU40NJPQMWIEKRYKWLIVR1&ll=37.50059,127.050775&v=20180604&radius=2500&limit=100'

In [21]:
# type your answer here
results = requests.get(url).json()
print('There are {} around Gangnam-gu.'.format(len(results['response']['groups'][0]['items'])))

There are 100 around Gangnam-gu.


In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
venues[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '53ad39bf498e55bfbfcbd8ca',
  'name': 'Fire Bell',
  'location': {'address': '강남구 선릉로72길 13',
   'lat': 37.500285014458655,
   'lng': 127.05227597008218,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.500285014458655,
     'lng': 127.05227597008218}],
   'distance': 136,
   'postalCode': '06199',
   'cc': 'KR',
   'city': '서울특별시',
   'state': '서울특별시',
   'country': '대한민국',
   'formattedAddress': ['강남구 선릉로72길 13',
    '대치4동',
    '서울특별시',
    '서울특별시',
    '06199',
    '대한민국']},
  'categories': [{'id': '4bf58dd8d48988d16c941735',
    'name': 'Burger Joint',
    'pluralName': 'Burger Joints',
    'shortName': 'Burgers',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-53ad39bf498e55bfb

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Fire Bell,Burger Joint,37.500285,127.052276
1,농민백암순대,Korean Restaurant,37.503792,127.053072
2,뽕나무쟁이족발,Bossam/Jokbal Restaurant,37.503248,127.052068
3,최인아책방,Bookstore,37.505994,127.048226
4,몰타참치,Sushi Restaurant,37.504960,127.055460


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Seoul

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=2500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.¶

In [27]:
# type your answer here
Seoul_venues = getNearbyVenues(names=neighborhoods['District'],
                                   latitudes=neighborhoods['Lat'],
                                   longitudes=neighborhoods['Long']
                                  )

Jungnang-gu
Gangnam-gu
Gangdong-gu
Gangbuk-gu
Gangseo-gu
Gwanak-gu
Gwangjin-gu
Guro-gu
Geumcheon-gu
Nowon-gu
Dobong-gu
Dongdaemun-gu
Mapo-gu
Seodaemun-gu
Seocho-gu
Seongdong-gu
Seongbuk-gu
Songpa-gu
Yangcheon-gu
Yeongdeungpo-gu
Yongsan-gu
Eunpyeong-gu
Jongno-gu
Jung-gu


In [28]:
print(Seoul_venues.shape)
Seoul_venues.head()

(2090, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jungnang-gu,37.597957,127.092666,Starbucks (스타벅스),37.597443,127.092570,Coffee Shop
1,Jungnang-gu,37.597957,127.092666,CGV Sangbong (CGV 상봉),37.597624,127.092461,Multiplex
2,Jungnang-gu,37.597957,127.092666,Costco Wholesale (코스트코),37.597334,127.090009,Warehouse Store
3,Jungnang-gu,37.597957,127.092666,맑은샘,37.591924,127.097330,Bunsik Restaurant
4,Jungnang-gu,37.597957,127.092666,Starbucks (스타벅스),37.596897,127.086460,Coffee Shop


In [29]:
Seoul_venues.groupby('District').count().reset_index()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dobong-gu,69,69,69,69,69,69
1,Dongdaemun-gu,100,100,100,100,100,100
2,Eunpyeong-gu,74,74,74,74,74,74
3,Gangbuk-gu,43,43,43,43,43,43
4,Gangdong-gu,100,100,100,100,100,100
5,Gangnam-gu,100,100,100,100,100,100
6,Gangseo-gu,95,95,95,95,95,95
7,Geumcheon-gu,38,38,38,38,38,38
8,Guro-gu,64,64,64,64,64,64
9,Gwanak-gu,100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(Seoul_venues['Venue Category'].unique())))

There are 191 uniques categories.


In [32]:
groupby_Seoul_venues=Seoul_venues.groupby('Venue Category').count()
groupby_Seoul_venues.sort_values('District', ascending=False).reset_index().head(20)

,Venue Category,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude
0,Coffee Shop,266,266,266,266,266,266
1,Korean Restaurant,230,230,230,230,230,230
2,Bakery,143,143,143,143,143,143
3,Café,106,106,106,106,106,106
4,BBQ Joint,87,87,87,87,87,87
5,Park,63,63,63,63,63,63
6,Chinese Restaurant,61,61,61,61,61,61
7,Ice Cream Shop,51,51,51,51,51,51
8,Japanese Restaurant,45,45,45,45,45,45
9,Fast Food Restaurant,42,42,42,42,42,42


# 2-2.New York City

In [37]:
NY_neighborhoods.head()

,Borough,Lat,Long
0,Bronx,40.850485,-73.840404
1,Manhattan,40.790087,-73.959829
2,Brooklyn,40.650104,-73.949582
3,Queens,40.652493,-73.791421
4,Staten Island,40.583456,-74.149605


#### Let's create a function to repeat the same process to all the neighborhoods in Seoul

In [38]:
# type your answer here
NY_venues = getNearbyVenues(names=NY_neighborhoods['Borough'],
                                   latitudes=NY_neighborhoods['Lat'],
                                   longitudes=NY_neighborhoods['Long']
                                  )

Bronx
Manhattan
Brooklyn
Queens
Staten Island


In [39]:
print(NY_venues.shape)
NY_venues.head()

(500, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,40.850485,-73.840404,Residence Inn by Marriott New York The Bronx a...,40.849325,-73.842534,Hotel
1,Bronx,40.850485,-73.840404,LA Fitness,40.849739,-73.841949,Gym / Fitness Center
2,Bronx,40.850485,-73.840404,Starbucks,40.851371,-73.844087,Coffee Shop
3,Bronx,40.850485,-73.840404,Zeppieri & Sons Italian Bakery,40.847119,-73.832057,Bakery
4,Bronx,40.850485,-73.840404,Empire Bagels,40.849392,-73.830527,Bagel Shop


In [40]:
NY_venues.groupby('District').count().reset_index()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,100,100,100,100,100,100
1,Brooklyn,100,100,100,100,100,100
2,Manhattan,100,100,100,100,100,100
3,Queens,100,100,100,100,100,100
4,Staten Island,100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [41]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 166 uniques categories.


In [42]:
groupby_NY_venues=NY_venues.groupby('Venue Category').count()
groupby_NY_venues.sort_values('District', ascending=False).reset_index().head(20)

,Venue Category,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude
0,Coffee Shop,22,22,22,22,22,22
1,Pizza Place,19,19,19,19,19,19
2,Caribbean Restaurant,15,15,15,15,15,15
3,Italian Restaurant,15,15,15,15,15,15
4,Park,14,14,14,14,14,14
5,Bakery,13,13,13,13,13,13
6,Airport Lounge,13,13,13,13,13,13
7,Clothing Store,11,11,11,11,11,11
8,Café,11,11,11,11,11,11
9,Cosmetics Shop,11,11,11,11,11,11


# 3.Analyze Each Neighborhood



## 3-1.Seoul

In [43]:
Seoul_venues[['Venue Category']].head()

,Venue Category
0,Coffee Shop
1,Multiplex
2,Warehouse Store
3,Bunsik Restaurant
4,Coffee Shop


In [44]:
Seoul_onehot=pd.get_dummies(Seoul_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Seoul_onehot['District'] = Seoul_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [Seoul_onehot.columns[-1]] + list(Seoul_onehot.columns[:-1])
Seoul_onehot = Seoul_onehot[fixed_columns]

Seoul_onehot.head()

,District,African Restaurant,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Bookstore,Bossam/Jokbal Restaurant,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bunsik Restaurant,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,College Theater,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Janguh Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Lottery Retailer,Lounge,Market,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Sake Bar,Samgyetang Restaurant,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Village,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Zoo
0,Jungnang-gu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Jungnang-gu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Jungnang-gu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Jungnang-gu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [45]:
Seoul_onehot.shape

(2090, 192)

### Next, let's group rows by District and by taking the mean of the frequency of occurrence of each category

In [46]:
Seoul_grouped = Seoul_onehot.groupby('District').mean().reset_index()
Seoul_grouped

,District,African Restaurant,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Bookstore,Bossam/Jokbal Restaurant,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bunsik Restaurant,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,College Theater,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Janguh Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Lottery Retailer,Lounge,Market,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Sake Bar,Samgyetang Restaurant,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Village,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Zoo
0,Dobong-gu,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.014493,0.043478,0.00,0.144928,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.014493,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.014493,0.028986,0.000000,0.014493,0.00,0.00,0.014493,0.00,0.072464,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.043478,0.00,0.000000,0.014493,0.072464,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.014493,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.014493,0.00,0.00,0.086957,0.00,0.00,0.014493,0.00,0.00,0.043478,0.086957,0.000000,0.00,0.014493,0.000000,0.00,0.00,0.00,0.00,0.00,0.014493,0.00,0.014493,0.014493,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.043478,0.00,0.00,0.014493,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.014493,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.014493,0.028986,0.00,0.014493,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.057971,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00
1,Dongdaemun-gu,0.00,0.00,0.00,0.00

In [47]:
Seoul_grouped.shape

(24, 192)

In [48]:
Seoul_grouped[Seoul_grouped['District'] == 'Gangnam-gu']

,District,African Restaurant,Airport Service,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Trail,Bistro,Bookstore,Bossam/Jokbal Restaurant,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bunsik Restaurant,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Cafeteria,College Gym,College Theater,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hostel,Hot Spring,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Janguh Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Korean Restaurant,Lottery Retailer,Lounge,Market,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,River,Rock Club,Sake Bar,Samgyetang Restaurant,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Village,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Zoo
5,Gangnam-gu,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.02,0.0,0.02,0.0,0.0,0.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.04,0.02,0.01,0.0,0.01,0.0,0.03,0.0,0.0,0.04,0.18,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
tmp=Seoul_grouped[Seoul_grouped['District'] == 'Gangnam-gu'].T.reset_index()
tmp[:10]

,index,5
0,District,Gangnam-gu
1,African Restaurant,0
2,Airport Service,0.01
3,American Restaurant,0
4,Aquarium,0
5,Arcade,0
6,Art Gallery,0
7,Art Museum,0
8,Arts & Crafts Store,0
9,Asian Restaurant,0


In [50]:
tmp2=tmp.iloc[1:]
tmp2[:10]

,index,5
1,African Restaurant,0
2,Airport Service,0.01
3,American Restaurant,0
4,Aquarium,0
5,Arcade,0
6,Art Gallery,0
7,Art Museum,0
8,Arts & Crafts Store,0
9,Asian Restaurant,0
10,Auto Dealership,0


In [51]:
num_top_venues = 5

for hood in Seoul_grouped['District']:
    print("----"+hood+"----")
    temp = Seoul_grouped[Seoul_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Dobong-gu----
                  venue  freq
0                Bakery  0.14
1     Korean Restaurant  0.09
2        Ice Cream Shop  0.09
3           Coffee Shop  0.07
4  Fast Food Restaurant  0.07


----Dongdaemun-gu----
               venue  freq
0  Korean Restaurant  0.18
1        Coffee Shop  0.17
2          BBQ Joint  0.08
3               Café  0.08
4             Bakery  0.07


----Eunpyeong-gu----
                  venue  freq
0           Coffee Shop  0.15
1                Bakery  0.14
2     Korean Restaurant  0.09
3        Ice Cream Shop  0.08
4  Fast Food Restaurant  0.05


----Gangbuk-gu----
               venue  freq
0        Coffee Shop  0.14
1  Korean Restaurant  0.09
2         Donut Shop  0.09
3             Bakery  0.07
4      Historic Site  0.07


----Gangdong-gu----
                  venue  freq
0           Coffee Shop  0.14
1                Bakery  0.14
2     Korean Restaurant  0.12
3  Fast Food Restaurant  0.08
4        Ice Cream Shop  0.07


----Gangnam-gu----
       

Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Seoul_venues_sorted = pd.DataFrame(columns=columns)
Seoul_venues_sorted['District'] = Seoul_grouped['District']

for ind in np.arange(Seoul_grouped.shape[0]):
    Seoul_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Seoul_grouped.iloc[ind, :], num_top_venues)

Seoul_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dobong-gu,Bakery,Ice Cream Shop,Korean Restaurant,Coffee Shop,Fast Food Restaurant,Trail,Donut Shop,BBQ Joint,Japanese Restaurant,Park
1,Dongdaemun-gu,Korean Restaurant,Coffee Shop,Café,BBQ Joint,Bakery,Chinese Restaurant,Fast Food Restaurant,Noodle House,Hotel,Bubble Tea Shop
2,Eunpyeong-gu,Coffee Shop,Bakery,Korean Restaurant,Ice Cream Shop,Fast Food Restaurant,BBQ Joint,Donut Shop,Mountain,Bookstore,Café
3,Gangbuk-gu,Coffee Shop,Korean Restaurant,Donut Shop,Trail,Ice Cream Shop,Bakery,Historic Site,Park,Café,Mountain
4,Gangdong-gu,Coffee Shop,Bakery,Korean Restaurant,Fast Food Restaurant,Ice Cream Shop,Donut Shop,Park,BBQ Joint,Market,Vietnamese Restaurant


# 3-2.New York City

In [82]:
NY_venues[['Venue Category']].head()

,Venue Category
0,Hotel
1,Gym / Fitness Center
2,Coffee Shop
3,Bakery
4,Bagel Shop


In [83]:
NY_onehot=pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['District'] = NY_venues['District'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,District,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Arcade,Arepa Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Store,Big Box Store,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Building,Burger Joint,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comic Shop,Community Center,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pub,Rental Car Location,Reservoir,Restaurant,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tibetan Restaurant,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bronx,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [84]:
NY_onehot.shape

(500, 167)

In [85]:
NY_grouped = NY_onehot.groupby('District').mean().reset_index()
NY_grouped

,District,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Arcade,Arepa Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Store,Big Box Store,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Building,Burger Joint,Butcher,Café,Campground,Candy Store,Caribbean Restaurant,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comic Shop,Community Center,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Empanada Restaurant,Exhibit,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pub,Rental Car Location,Reservoir,Restaurant,Roof Deck,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tibetan Restaurant,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Bronx,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.04,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.05,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.01,0.04,0.00,0.01,0.03,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.04,0.03,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.09,0.00,0.01,0.00,0.00,0.02,0.01,0.01,0.01,0.01,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.01,0.13,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.03,0.00,0.00,0.01,0.00,0.00,0.03,0.00,0.02,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Brooklyn,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.04,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.01,0.01,0.00,0.00,0.06,0.00,0.00,0.13,0.00,0.01,0.00,0.03,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.01,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.02,0.01,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01,0.02,0.01,0.01,0.00,0.00,0.00,0.04,0.00,0.00,0.02,0.00,0.02,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.0

In [86]:
num_top_venues = 5

for hood in NY_grouped['District']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bronx----
                venue  freq
0         Pizza Place  0.13
1  Italian Restaurant  0.09
2         Coffee Shop  0.05
3              Bakery  0.04
4                 Gym  0.04


----Brooklyn----
                  venue  freq
0  Caribbean Restaurant  0.13
1                  Café  0.06
2                Bakery  0.04
3                Garden  0.04
4                  Park  0.04


----Manhattan----
        venue  freq
0        Park  0.07
1     Exhibit  0.06
2  Art Museum  0.04
3      Bakery  0.03
4      Garden  0.03


----Queens----
                 venue  freq
0       Airport Lounge  0.13
1  Rental Car Location  0.07
2           Donut Shop  0.06
3       Cosmetics Shop  0.06
4          Coffee Shop  0.05


----Staten Island----
                    venue  freq
0          Clothing Store  0.08
1             Coffee Shop  0.07
2          Cosmetics Shop  0.04
3  Furniture / Home Store  0.03
4                Pharmacy  0.03




In [87]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
NY_venues_sorted = pd.DataFrame(columns=columns)
NY_venues_sorted['Borough'] = NY_grouped['District']

for ind in np.arange(NY_grouped.shape[0]):
    NY_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

NY_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Pizza Place,Italian Restaurant,Coffee Shop,Bakery,Gym,Deli / Bodega,Spanish Restaurant,Supermarket,Gym / Fitness Center,Mexican Restaurant
1,Brooklyn,Caribbean Restaurant,Café,Garden,Park,Bakery,Cocktail Bar,Pizza Place,Grocery Store,Thai Restaurant,Bar
2,Manhattan,Park,Exhibit,Art Museum,Yoga Studio,Plaza,Bakery,Garden,Bookstore,American Restaurant,Coffee Shop
3,Queens,Airport Lounge,Rental Car Location,Cosmetics Shop,Donut Shop,Coffee Shop,Hotel,Sandwich Place,Southern / Soul Food Restaurant,Ice Cream Shop,Deli / Bodega
4,Staten Island,Clothing Store,Coffee Shop,Cosmetics Shop,Pharmacy,Furniture / Home Store,Department Store,Toy / Game Store,Burger Joint,Spa,Electronics Store


# 4.Cluster Neighborhoods



## 4-1.Seoul

Run k-means to cluster the neighborhood into 5 clusters.

In [60]:
# set number of clusters
kclusters = 5

Seoul_grouped_clustering = Seoul_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Seoul_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 0, 0, 4, 1, 0], dtype=int32)

In [61]:
# add clustering labels
Seoul_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Seoul_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Seoul_merged = Seoul_merged.join(Seoul_venues_sorted.set_index('District'), on='District')

Seoul_merged.head() # check the last columns!

,District,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jungnang-gu,37.597957,127.092666,1,Bakery,Coffee Shop,Ice Cream Shop,Supermarket,Fast Food Restaurant,Donut Shop,Korean Restaurant,Multiplex,Steakhouse,Bunsik Restaurant
1,Gangnam-gu,37.500590,127.050775,0,Korean Restaurant,Coffee Shop,Bakery,BBQ Joint,Burger Joint,Hotel,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Dessert Shop
2,Gangdong-gu,37.547070,127.141903,1,Coffee Shop,Bakery,Korean Restaurant,Fast Food Restaurant,Ice Cream Shop,Donut Shop,Park,BBQ Joint,Market,Vietnamese Restaurant
3,Gangbuk-gu,37.640495,127.009235,1,Coffee Shop,Korean Restaurant,Donut Shop,Trail,Ice Cream Shop,Bakery,Historic Site,Park,Café,Mountain
4,Gangseo-gu,37.558340,126.832201,0,Coffee Shop,Korean Restaurant,Bakery,Noodle House,Multiplex,Supermarket,Ice Cream Shop,BBQ Joint,Fast Food Restaurant,Donut Shop


In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Seoul_merged['Lat'], Seoul_merged['Long'], Seoul_merged['District'], Seoul_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4-2.New York

In [88]:
# set number of clusters
kclusters = 5

NY_grouped_clustering = NY_grouped.drop('District', 1)

# run k-means clustering
NY_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
NY_kmeans.labels_[0:10] 

array([3, 4, 1, 2, 0], dtype=int32)

In [89]:
NY_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Pizza Place,Italian Restaurant,Coffee Shop,Bakery,Gym,Deli / Bodega,Spanish Restaurant,Supermarket,Gym / Fitness Center,Mexican Restaurant
1,Brooklyn,Caribbean Restaurant,Café,Garden,Park,Bakery,Cocktail Bar,Pizza Place,Grocery Store,Thai Restaurant,Bar
2,Manhattan,Park,Exhibit,Art Museum,Yoga Studio,Plaza,Bakery,Garden,Bookstore,American Restaurant,Coffee Shop
3,Queens,Airport Lounge,Rental Car Location,Cosmetics Shop,Donut Shop,Coffee Shop,Hotel,Sandwich Place,Southern / Soul Food Restaurant,Ice Cream Shop,Deli / Bodega
4,Staten Island,Clothing Store,Coffee Shop,Cosmetics Shop,Pharmacy,Furniture / Home Store,Department Store,Toy / Game Store,Burger Joint,Spa,Electronics Store


In [90]:
NY_kmeans.labels_

array([3, 4, 1, 2, 0], dtype=int32)

In [91]:
# add clustering labels
NY_venues_sorted.insert(0, 'Cluster Labels', NY_kmeans.labels_)

NY_merged = NY_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
NY_merged = NY_merged.join(NY_venues_sorted.set_index('Borough'), on='Borough')

NY_merged.head() # check the last columns!

,Borough,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,40.850485,-73.840404,3,Pizza Place,Italian Restaurant,Coffee Shop,Bakery,Gym,Deli / Bodega,Spanish Restaurant,Supermarket,Gym / Fitness Center,Mexican Restaurant
1,Manhattan,40.790087,-73.959829,1,Park,Exhibit,Art Museum,Yoga Studio,Plaza,Bakery,Garden,Bookstore,American Restaurant,Coffee Shop
2,Brooklyn,40.650104,-73.949582,4,Caribbean Restaurant,Café,Garden,Park,Bakery,Cocktail Bar,Pizza Place,Grocery Store,Thai Restaurant,Bar
3,Queens,40.652493,-73.791421,2,Airport Lounge,Rental Car Location,Cosmetics Shop,Donut Shop,Coffee Shop,Hotel,Sandwich Place,Southern / Soul Food Restaurant,Ice Cream Shop,Deli / Bodega
4,Staten Island,40.583456,-74.149605,0,Clothing Store,Coffee Shop,Cosmetics Shop,Pharmacy,Furniture / Home Store,Department Store,Toy / Game Store,Burger Joint,Spa,Electronics Store


In [92]:
# create map
map_clusters = folium.Map(location=[NY_latitude, NY_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Lat'], NY_merged['Long'], NY_merged['Borough'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5.Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

# 5-1.Seoul

In [93]:
#Seoul_merged.loc[Seoul_merged['Cluster Labels'] == 0, Seoul_merged.columns[[1] + list(range(5, Seoul_merged.shape[1]))]]
S0=Seoul_merged.loc[Seoul_merged['Cluster Labels'] == 0, Seoul_merged.columns[[0] + list(range(4, Seoul_merged.shape[1]))] ]
S0

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Gangnam-gu,Korean Restaurant,Coffee Shop,Bakery,BBQ Joint,Burger Joint,Hotel,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Dessert Shop
4,Gangseo-gu,Coffee Shop,Korean Restaurant,Bakery,Noodle House,Multiplex,Supermarket,Ice Cream Shop,BBQ Joint,Fast Food Restaurant,Donut Shop
5,Gwanak-gu,Coffee Shop,Korean Restaurant,Bakery,BBQ Joint,Café,Japanese Restaurant,Chinese Restaurant,Vietnamese Restaurant,Burger Joint,Ice Cream Shop
9,Nowon-gu,Coffee Shop,Korean Restaurant,Ice Cream Shop,Fast Food Restaurant,Trail,Mountain,Bakery,Noodle House,Lottery Retailer,Donut Shop
11,Dongdaemun-gu,Korean Restaurant,Coffee Shop,Café,BBQ Joint,Bakery,Chinese Restaurant,Fast Food Restaurant,Noodle House,Hotel,Bubble Tea Shop
14,Seocho-gu,Coffee Shop,Korean Restaurant,BBQ Joint,Bakery,Noodle House,Café,Park,Burger Joint,Italian Restaurant,Concert Hall
16,Seongbuk-gu,Korean Restaurant,Coffee Shop,Café,Bakery,Noodle House,Donut Shop,Chinese Restaurant,Ice Cream Shop,Dessert Shop,BBQ Joint
17,Songpa-gu,Korean Restaurant,Coffee Shop,Café,Bakery,Italian Restaurant,Park,BBQ Joint,Chinese Restaurant,Hotel,Trail
19,Yeongdeungpo-gu,Korean Restaurant,Coffee Shop,Park,Japanese Restaurant,Hotel,Chinese Restaurant,Café,BBQ Joint,Bakery,Burger Joint
20,Yongsan-gu,Korean Restaurant,Coffee Shop,BBQ Joint,Cocktail Bar,Museum,Pizza Place,Noodle House,Lounge,Café,Mexican Restaurant


In [94]:
S1=Seoul_merged.loc[Seoul_merged['Cluster Labels'] == 1,Seoul_merged.columns[[0] + list(range(4, Seoul_merged.shape[1]))]]
S1

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jungnang-gu,Bakery,Coffee Shop,Ice Cream Shop,Supermarket,Fast Food Restaurant,Donut Shop,Korean Restaurant,Multiplex,Steakhouse,Bunsik Restaurant
2,Gangdong-gu,Coffee Shop,Bakery,Korean Restaurant,Fast Food Restaurant,Ice Cream Shop,Donut Shop,Park,BBQ Joint,Market,Vietnamese Restaurant
3,Gangbuk-gu,Coffee Shop,Korean Restaurant,Donut Shop,Trail,Ice Cream Shop,Bakery,Historic Site,Park,Café,Mountain
7,Guro-gu,Bakery,Coffee Shop,Donut Shop,Park,Korean Restaurant,Ice Cream Shop,Hotel,Market,Fast Food Restaurant,Multiplex
10,Dobong-gu,Bakery,Ice Cream Shop,Korean Restaurant,Coffee Shop,Fast Food Restaurant,Trail,Donut Shop,BBQ Joint,Japanese Restaurant,Park
18,Yangcheon-gu,Coffee Shop,Korean Restaurant,Bakery,Park,Chinese Restaurant,Ice Cream Shop,Fast Food Restaurant,Sushi Restaurant,Dessert Shop,Steakhouse
21,Eunpyeong-gu,Coffee Shop,Bakery,Korean Restaurant,Ice Cream Shop,Fast Food Restaurant,BBQ Joint,Donut Shop,Mountain,Bookstore,Café


In [95]:
S2=Seoul_merged.loc[Seoul_merged['Cluster Labels'] == 2,Seoul_merged.columns[[0] + list(range(4, Seoul_merged.shape[1]))]]
S2

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Gwangjin-gu,Coffee Shop,Café,Chinese Restaurant,Park,Korean Restaurant,BBQ Joint,Market,Bunsik Restaurant,Performing Arts Venue,Noodle House
12,Mapo-gu,Coffee Shop,BBQ Joint,Park,Korean Restaurant,Café,Chinese Restaurant,Sake Bar,Italian Restaurant,Japanese Restaurant,Bakery
13,Seodaemun-gu,Coffee Shop,Café,Korean Restaurant,Chinese Restaurant,Japanese Restaurant,Bagel Shop,Tea Room,Cocktail Bar,Sake Bar,Sandwich Place
15,Seongdong-gu,Coffee Shop,Café,Korean Restaurant,BBQ Joint,Park,Vietnamese Restaurant,Restaurant,Bakery,Electronics Store,Other Great Outdoors


In [97]:
S3=Seoul_merged.loc[Seoul_merged['Cluster Labels'] == 3,Seoul_merged.columns[[0] + list(range(4, Seoul_merged.shape[1]))]]
S3

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Jongno-gu,Korean Restaurant,Café,Historic Site,Italian Restaurant,Coffee Shop,Tea Room,Palace,Art Gallery,Japanese Restaurant,Art Museum
23,Jung-gu,Hotel,Korean Restaurant,Café,Coffee Shop,Plaza,Theater,Chinese Restaurant,Historic Site,Bookstore,Noodle House


In [98]:
S4=Seoul_merged.loc[Seoul_merged['Cluster Labels'] == 4,Seoul_merged.columns[[0] + list(range(4, Seoul_merged.shape[1]))]]
S4

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Geumcheon-gu,Coffee Shop,Fast Food Restaurant,Korean Restaurant,Trail,BBQ Joint,Supermarket,Outlet Store,Gym,Bakery,Café


# 5-2.New York City

In [99]:
N1=NY_merged.loc[NY_merged['Cluster Labels'] == 0, NY_merged.columns[[0] + list(range(4, NY_merged.shape[1]))] ]
N1

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Staten Island,Clothing Store,Coffee Shop,Cosmetics Shop,Pharmacy,Furniture / Home Store,Department Store,Toy / Game Store,Burger Joint,Spa,Electronics Store


In [100]:
N2=NY_merged.loc[NY_merged['Cluster Labels'] == 1, NY_merged.columns[[0] + list(range(4, NY_merged.shape[1]))] ]
N2

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Manhattan,Park,Exhibit,Art Museum,Yoga Studio,Plaza,Bakery,Garden,Bookstore,American Restaurant,Coffee Shop


In [101]:
N3=NY_merged.loc[NY_merged['Cluster Labels'] == 2, NY_merged.columns[[0] + list(range(4, NY_merged.shape[1]))] ]
N3

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Queens,Airport Lounge,Rental Car Location,Cosmetics Shop,Donut Shop,Coffee Shop,Hotel,Sandwich Place,Southern / Soul Food Restaurant,Ice Cream Shop,Deli / Bodega


In [102]:
N4=NY_merged.loc[NY_merged['Cluster Labels'] == 3, NY_merged.columns[[0] + list(range(4, NY_merged.shape[1]))] ]
N4

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Pizza Place,Italian Restaurant,Coffee Shop,Bakery,Gym,Deli / Bodega,Spanish Restaurant,Supermarket,Gym / Fitness Center,Mexican Restaurant


In [103]:
N5=NY_merged.loc[NY_merged['Cluster Labels'] == 4, NY_merged.columns[[0] + list(range(4, NY_merged.shape[1]))] ]
N5


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Brooklyn,Caribbean Restaurant,Café,Garden,Park,Bakery,Cocktail Bar,Pizza Place,Grocery Store,Thai Restaurant,Bar
